Procesamiento de datos del segundo fomulario. El que hice en Exactas y Fadu con aprox 15 personas en casa uno. 

Los datos originales estan aca:

https://github.com/IoniUTDT/Magia/tree/master/PilotoFacuFadu

El proyecto Magia es un proyecto que busca medir como responde la gente frente a una situacion novedosa y dificil de explicar en terminos fisicos. La idea es hacer un "truco" (el del buzo en una botella de agua) donde no es obvio porque el globo sube o baja, disfrazado de algo "sorprendente" y pedirle a la gente que complete una encuesta sobre que cree que esta pasando. Buscamos ver cuanto inventa la gente y con que sesgo en funcion de su formacion profesional. 

Los datos de la encuesta son levantados automaticamente mediante el sotware sdaps que genera los formularios y despues mediante escaneo los procesa en bloque sin necesidad de pasar los datos a manos a una compu. Del programa se puede exportar un csv con los datos en bruto. Donde a cada entrada le corresponde una encuesta completada. 

In [11]:
%load_ext autoreload
%autoreload 2
import pandas
from IPython.display import display
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


En el siguiente bloque cargamos los datos y los procesamos para que sean mas legibles. Para eso hay que interpretar el csv donde viene guardados. El [csv](./DataValidada_PilotoKonex.csv) se genero automaticamente por el spdaps a partir de escanear el [archivo](./questionnaire.pdf), donde como se puede ver, hay secciones, preguntas y opciones multiples. Cada seccion esta indicada en el encabezado por un primer numero, cada preguntas por un segundo numero y cada opcion por un tercero unidos con un _ luego cada fila representa una entrada donde un 1 representa seleccionado en el caso de las opciones multiples o el valor seleccionado en caso de que se tratara de una escala. 

In [16]:
data = pandas.read_csv('./data_Micro.csv')
with open('Codigos.txt') as f:
    lineas = f.readlines()

grupos = []
subGrupo = []

for linea in lineas:
    if linea == "\n":
        grupos.append(subGrupo)
        subGrupo = []
    else:
        subGrupo.append(linea.split('\n')[0])
if not subGrupo == []:
    grupos.append(subGrupo)

for grupo in grupos:
    encabezado = grupo[0]
    inicio = grupo[1]
    opciones = grupo[2::]
    if len(opciones)>0:
        titulos = [inicio+'_'+str(indice) for indice in range(len(opciones))]
    else:
        titulos = [grupo[1]]
    mapeo = {x:y for x,y in zip(titulos,opciones)}
    if len(titulos) > 1:
        data[encabezado] = data[titulos].idxmax(axis=1)
        data[encabezado].replace(mapeo, inplace=True)
        data.loc[data[titulos].sum(axis=1)<1, encabezado] = 0
        data.loc[data[titulos].sum(axis=1)>1, encabezado] = -1
    else:
        data[encabezado] = data[titulos]
    data = data.drop(titulos, 1)

In [17]:
display(data.head())

,questionnaire_id,global_id,2_2_1,2_2_2,2_2_3,2_2_4,2_2_5,2_2_6,2_2_7,2_2_8,...,Calor1,Telequinesis1,Magnetismo1,Cosmica1,Cuantica1,Presion1,Aura1,Electricidad1,Poderes1,Magnetismo
0,['37441'],1003,7,7,1,7,7,7,7,7,...,0,0,Podria,0,0,0,0,0,0,0
1,['55027'],1003,3,7,2,6,6,4,7,3,...,0,0,Es,0,0,0,0,Podria,0,0
2,['23839'],1003,3,7,7,7,7,4,7,4,...,-1,0,0,0,0,Podria,0,0,0,0
3,['55177'],1003,7,7,7,7,7,1,7,7,...,0,0,0,0,0,Es,0,0,0,0
4,['50273'],1003,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
